# Analysis for Performance and Energy Consumption

## Extract information

In [12]:
# Libraries

import sys
import os
import glob
import webbrowser
import natsort
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MaxAbsScaler
from IPython.display import display, HTML


In [13]:
# Directory
actual_directory = os.getcwd() + '/'  

python_releaseDates = {
    'Python 3.13.0a0': '2023-06-07',
    'Python 3.12.0b1': '2023-05-22',
    'Python 3.11.3': '2023-04-05',
    'Python 3.10.11': '2023-04-05',
    'Python 3.9.16': '2022-12-06',
    'Python 3.8.16': '2022-12-06',
    'Python 3.7.16': '2022-12-06',
    'Python 3.6.15': '2021-09-04',
    'Python 3.5.10': '2020-09-05',
    'Python 3.4.10': '2019-03-18',
    'Python 3.3.7': '2017-09-19',
    'Python 3.2.6': '2014-10-11',
    'Python 3.1.5': '2012-04-09',
    'Python 3.0.1': '2009-02-13',
    'Python 2.7.18': '2020-04-20',
    'Python 2.6.9': '2013-10-29',
    'Python 2.5.6': '2011-05-26',
}

cplusplus_releaseDates = {
    'g++-4.4 4.4.7': '2012-03-13',
    'g++-4.6 4.6.4': '2013-04-12',
    'g++-4.7 4.7.4': '2014-06-12',
    'g++-4.8 4.8.5': '2015-06-23',
    'g++-4.9 4.9.3': '2015-06-26',
    'g++-5 5.5.0': '2017-10-10',
    'g++-6 6.5.0': '2018-10-26',
    'g++-7 7.5.0': '2019-11-14',
    'g++-8 8.5.0': '2021-05-14',
    'g++-8 ': '2021-05-14',
    'g++-9 9.5.0': '2022-05-27',
    'g++-10 10.4.0': '2022-06-28',
    'g++-10 10.5.0': '2023-07-07',
    'g++-11 11.4.0': '2023-05-29',
    'g++-12 12.3.0': '2023-05-08',
    'g++-13 13.1.0': '2023-04-26',
}

java_releaseDates = {
    '1.8.0_362': '2023-04-18',
    '9.0.4': '2018-01-16',
    '10.0.2': '2018-07-17',
    '11.0.19': '2020-10-20',
    '12.0.2': '2019-07-16',
    '13.0.2': '2020-01-14',
    '14.0.2': '2020-07-14',
    '15.0.2': '2021-01-19',
    '16.0.2': '2021-07-20',
    '17.0.7': '2023-04-18',
    '18.0.2-ea': '2022-07-19',
    '19.0.2': '2023-01-17',
    '20.0.2': '2023-07-18',
}

js_releaseDates = {
    '20.5.1': '2023-08-09',
    '19.9.0': '2023-04-10',
    '18.17.1': '2023-08-08',
    '17.9.1': '2022-06-01',
    '16.20.2': '2023-08-08',
    '15.14.0': '2021-04-06',
    '14.21.3': '2023-02-16',
    '13.14.0': '2020-04-29',
    '12.22.12': '2022-04-05',
    '11.15.0': '2019-04-30',
    '10.24.1': '2021-04-06',
    '9.11.2': '2018-06-12',
    '8.17.0': '2019-12-17',
    '7.10.1': '2017-07-11',
    '6.17.1': '2019-04-03',
    '5.12.0': '2016-06-23',
    '4.9.1': '2018-03-29',
    '3.3.1': '2015-09-15',
    '2.5.0': '2015-07-28',
    '1.8.4': '2015-07-09',
    '0.12.18': '2017-02-22',
    '0.10.48': '2016-10-18',
    '0.8.28': '2014-07-31'
}

# Function to get release date for a given Python version
def get_release_date(version):
    if language == 'python':
        return python_releaseDates.get(version, 'Unknown')
    elif language == 'c++':
        return cplusplus_releaseDates.get(version, 'Unknown')
    elif language == 'java':
        return java_releaseDates.get(version, 'Unknown')
    elif language == 'js':
        return js_releaseDates.get(version, 'Unknown')

# Function to convert "g", "m" or KiB to "byte"
def convert_g_to_byte(value):
    value_str = str(value)
    if value_str[:-1] == "":
        Byte_value = 0
    elif value_str[-1].lower() == 'g':
        g_value = float(value_str[:-1])
        Byte_value = g_value * 1024 * 1024 * 1024  # 1 giga = 1,000,000,000
    elif value_str[-1].lower() == 'm':
        m_value = float(value_str[:-1])
        Byte_value = m_value * 1024 * 1024 # 1 giga = 1,000,000
    else:
        k_value = float(value_str)
        Byte_value = k_value * 1024 # 1 kilo = 1,000
    return int(Byte_value)

def convert_toUnit(column):
    column = column.apply(convert_g_to_byte)
    column = pd.to_numeric(column, errors='coerce')
    return column
    
def Data_normalized(df, tool):
    df_data = df[['version', 'release_date', 'path', 'appplication']]
    if tool == "turbostat": 
        df_metric = df.loc[:, ~df.columns.isin(['version', 'release_date', 'path', 'appplication'])]
    elif tool == "top":
        df_metric = df[['virt', 'res', 'shr', 'percent_cpu', 'percent_mem',
                    'nTH', 'P', 'SWAP', 'CODE', 'DATA', 'nMaj',
                    'nDRT', 'USED']]

    transformer = MaxAbsScaler().fit(df_metric)
    scaled = transformer.transform(df_metric)

    df_norm = pd.DataFrame(scaled, columns=df_metric.columns)
    df = pd.concat([df_data,df_norm.reindex(df_data.index)], axis=1)
    return df

# Function to extract information in every file
def from_CSVfile(file, directory, tool):
     # Read CSV file
    df = pd.read_csv(file)
    if language == 'js': df['version'] = df['version'].str.replace('v', '')
    
    # New column 'release_date' as the second 
    df['release_date'] = df['version'].apply(get_release_date)
    df.insert(1, 'release_date', df.pop('release_date'))

    # Convert date into datetime
    df['release_date'] = pd.to_datetime(df['release_date'])

    # New column 'path' as the third
    df['path'] = directory
    df.insert(2, 'path', df.pop('path'))

    # Clean and remove the unnecessary rows
    df.replace(to_replace='-', value=0, inplace=True)

    # Apply the conversion function to the DataFrame column
    if tool == "top":
        df = df.dropna(subset=['command'])
        df['virt'] = convert_toUnit(df['virt'])
        df['res'] = convert_toUnit(df['res'])
        df['shr'] = convert_toUnit(df['shr'])
        df['CODE'] = convert_toUnit(df['CODE'])
        df['DATA'] = convert_toUnit(df['DATA'])
        df['SWAP'] = convert_toUnit(df['SWAP'])
        df['USED'] = convert_toUnit(df['USED'])

    # Changes in the 'version' column
    if language == 'python': df['version'] = df['version'].str.replace('Python ', '')
    if language == 'c++': df['version'] = df['version'].str.split().str[0]
    #if language == 'js': df['version'] = df['version'].apply(lambda x: 'v' + x)

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #     print(df)

    return df

# Extract all information from ALL files
def from_CSVfiles(tool, norm):

    all_df = pd.DataFrame()

    extra = ''
    if tool == "turbostat": extra = "_performance"

    list_directories = next(os.walk(language))[1]
    list_directories.sort()

    for directory_name in list_directories:
        if directory_name != "older" and directory_name != "test" and directory_name != "general_plots":
            path=language + '/' + directory_name + '/' + tool + '/'
            if tool != "top": path = path + tool + extra
            if tool == "top":

                df = pd.DataFrame()
                list_files = os.listdir(path)

                # Get list of all files only in the given directory
                list_files = natsort.natsorted(list_files)

                for file_name in list_files:
                    if file_name.startswith("temp_top_data_") and file_name.endswith('.csv'):
                        df_Top = from_CSVfile(path + file_name, directory_name, tool)
                        df = pd.concat([df, df_Top])

                df.to_csv(path + "top_data_allVersions.csv", index=False)
            else:
                df = from_CSVfile(path + '_data_allVersions.csv', directory_name, tool)
                if norm: df = Data_normalized(df, tool)
            all_df = pd.concat([all_df, df])

    return all_df

In [14]:
language = "js"

df = from_CSVfiles("perf", norm=False)

FileNotFoundError: [Errno 2] No such file or directory: 'js/general_plots_v1/perf/perf_data_allVersions.csv'

In [ ]:
def custom_hover(program, x, y, mean, diff, percent_diff):
    percent_difffromMean = 0 if mean == 0 else 100 * (y/mean)
    return f'Version: {x}<br>Program: {program}<br><b><i>Percentage Difference:</i> {percent_diff:.2f}%</b><br>Difference: {diff:.2f}<br><b><i>Percentage Difference from Mean:</i> {percent_difffromMean:.2f}%</b><br>Difference From Mean: {y:.2f}<br>Mean: {mean:.2f}<br>'


In [ ]:
language = "js"
x_data = "version"
y_data = "virt"
color_data = "path"
type = "barTop"
diff = True

df = from_CSVfiles("top", norm=False)


if type == "barTop": df = df.groupby([x_data,color_data], sort=False)[[y_data]].mean().reset_index()
if type == "lineTop": df = df.groupby([x_data,color_data], sort=False)[[y_data]].mean().reset_index()

mean_per_program = df.groupby(color_data)[y_data].mean()
max_per_program = df.groupby(color_data)[y_data].idxmax()
max_energy_df = df.loc[max_per_program, [color_data, x_data, y_data]]
display(max_energy_df)
min_per_program = df.groupby(color_data)[y_data].idxmin()
min_energy_df = df.loc[min_per_program, [color_data, x_data, y_data]]
display(min_energy_df)
df['DifferenceFromMean'] = df.apply(lambda row: row[y_data] - mean_per_program[row[color_data]], axis=1)

# Create grouped bar traces for each program
bar_traces = []
for program in df[color_data].unique():
    program_df = df[df[color_data] == program]
    mean = mean_per_program[program]
    program_df['Difference'] = program_df[y_data].diff()
    program_df['Percentage_Difference'] = program_df[y_data].pct_change()*100
    program_df['Percentage_DifferenceFromMean'] = program_df.apply(lambda row: 0 if (mean == 0) else 100 * (row["DifferenceFromMean"] / mean), axis=1)
    hover_texts = [custom_hover(program, x, y, mean, diff, percent_diff) for x, y, diff, percent_diff in zip(program_df[x_data], program_df['DifferenceFromMean'],program_df['Difference'],program_df['Percentage_Difference'])]
    if diff:
        min = program_df[program_df['Difference'] == program_df['Difference'].min()]
        max = program_df[program_df['Difference'] == program_df['Difference'].max()]
        bar_trace = go.Bar(x=program_df[x_data], y=program_df['Difference'],
                            name=f'{program} - Mean: { format(mean_per_program[program], ".2f")}',
                            hovertemplate=hover_texts,
                            text=program_df["Percentage_Difference"])
    else:
        min = program_df[program_df['DifferenceFromMean'] == program_df['DifferenceFromMean'].min()]
        max = program_df[program_df['DifferenceFromMean'] == program_df['DifferenceFromMean'].max()]
        bar_trace = go.Bar(x=program_df[x_data], y=program_df['DifferenceFromMean'],
                           name=f'{program} - Mean: { format(mean_per_program[program], ".2f")}',
                           hovertemplate=hover_texts,
                           text=program_df["Percentage_DifferenceFromMean"])
    
    bar_traces.append(bar_trace)
    # bar_traces.append(median_trace)

layout = go.Layout(title='Comparison of ' + y_data,
                       xaxis=dict(title=x_data),
                       yaxis=dict(title='Difference from Mean of ' + y_data, zeroline=False))
        
fig = go.Figure(data=bar_traces, layout=layout)
if language == 'js': fig.update_xaxes(categoryorder='array', categoryarray= ['0.8.28', '0.10.48', '0.12.18', '1.8.4', '2.5.0', '3.3.1', '4.9.1', '5.12.0', '6.17.1', '7.10.1', '8.17.0', '9.11.2', '10.24.1', '11.15.0', '12.22.12', '13.14.0', '14.21.3', '15.14.0', '16.20.2', '17.9.1', '18.17.1', '19.9.0', '20.5.1'])

# for _, row in max_energy_df.iterrows():
#     fig.add_annotation(
#         text=f"Max: {row[y_data]} ({row[color_data]} {row[x_data]})",
#         x=row[x_data],
#         y=row[y_data],
#     )

fig.show()


path version          virt
8   binaryTrees_21_original_v1  19.9.0  6.219736e+09
12  binaryTrees_21_original_v7  8.17.0  1.806563e+09
36           binaryTrees_21_v7  9.11.2  1.674272e+09
69              nbody_50000000  19.9.0  1.014264e+09
84     nbody_50000000_original  19.9.0  1.014096e+09

path  version          virt
3   binaryTrees_21_original_v1  14.21.3  2.533270e+09
14  binaryTrees_21_original_v7  10.24.1  9.451342e+08
26           binaryTrees_21_v7  0.10.48  8.989883e+08
60              nbody_50000000  10.24.1  5.618975e+08
75     nbody_50000000_original  10.24.1  5.618770e+08

/tmp/ipykernel_107093/3395108535.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_107093/3395108535.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_107093/3395108535.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

In [ ]:
import pandas as pd
import plotly.express as px

# Create a sample DataFrame (replace this with your data)
data = {
    'Category': ['Category A', 'Category B', 'Category C', 'Category D'],
    'Value1': [10, 15, 7, 12],
    'Value2': [5, 9, 4, 8],
    'Value3': [8, 12, 6, 10],
    'Value4': [6, 11, 5, 9],
    'LineValue': [3, 6, 4, 5]  # New column for the line plot
}

df = pd.DataFrame(data)

# Melt the DataFrame
df_melted = pd.melt(df, id_vars=['Category'], var_name='Variable', value_name='Value')

# Create the bar plot
fig = px.bar(df_melted, x='Category', y='Value', color='Variable', title='Bar Plot with Multiple Variables')

# Customize the layout if needed (e.g., axis labels, title, legend, etc.)
fig.update_layout(
    xaxis_title='Categories',
    yaxis_title='Values',
    legend_title='Variables'
)

# Create the line plot trace
line_trace = px.line(df, x='Category', y='LineValue', title='Line Plot', labels={'LineValue': 'Line Data'})

# Add the line trace to the existing figure
fig.add_trace(line_trace.data[0])

# Show the combined plot
fig.show()


In [ ]:
df

Category  Value1  Value2  Value3  Value4
0  Category A      10       5       8       6
1  Category B      15       9      12      11
2  Category C       7       4       6       5
3  Category D      12       8      10       9

In [ ]:
df_melted

Category Variable  Value
0   Category A   Value1     10
1   Category B   Value1     15
2   Category C   Value1      7
3   Category D   Value1     12
4   Category A   Value2      5
5   Category B   Value2      9
6   Category C   Value2      4
7   Category D   Value2      8
8   Category A   Value3      8
9   Category B   Value3     12
10  Category C   Value3      6
11  Category D   Value3     10
12  Category A   Value4      6
13  Category B   Value4     11
14  Category C   Value4      5
15  Category D   Value4      9

In [18]:
div_html = ''
flag = True

for i in range(0,5):

    if flag:
        div_html = div_html + '''
                    <div class="row">
            '''
    div_html = div_html + "<column>Hola"
    if not flag:
        div_html = div_html + '''</div>''' 
        flag = True
    else:
        flag = False
if not flag:
    div_html = div_html + '''</div>''' 

print(div_html)


                    <div class="row">
            <column>Hola<column>Hola</div>
                    <div class="row">
            <column>Hola<column>Hola</div>
                    <div class="row">
            <column>Hola</div>
